## 1. Download a local copy of an opensourced LLM

In [3]:
import os
import torch
from huggingface_hub import hf_hub_download

<font size="2">Prerequisites: create access token on huggingFace via profile settings and set this token in your system user variables </font>

In [4]:
HUGGING_FACE_API_KEY = os.environ.get("HUGGING_FACE_API_KEY") 

The opensource model [**Distilbert**](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english/blob/main/README.md) has a transformer architecture, meaning it uses self-attention mechanisms to understand the relationships between words in a sentence, regardless of their distance. It is a smaller and faster version of the well-known BERT (Bidirectional Encoder Representations from Transformers) model. It was created by Hugging Face, a company known for its open-source natural language processing (NLP) tools  and libraries.  
* DistilBERT retains the core functionalities of BERT while being more efficient in terms of parameters and computational resources.
* It is trained on SST-2, which stands for Stanford Sentiment Treebank. SST-2 is a widely used dataset for sentiment analysis tasks in Natural Language Processing (NLP).
* [SST-2 contains a corpus of 11,855 sentences extracted from movie review    Sentiment Labels: Each sentence is assigned a sentiment of positive or negative](https://huggingface.co/datasets/sst2)

In [5]:
model_id = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
filenames = ["config.json","model.safetensors","pytorch_model.bin","rust_model.ot","tf_model.h5","tokenizer_config.json","vocab.txt"]

<font size="2"> Donwload the model so it runs locally </font>

In [6]:
for filename in filenames: 
    downloaded_model_path = hf_hub_download(
            repo_id = model_id,
            filename = filename,
            token = HUGGING_FACE_API_KEY
    )
    print (downloaded_model_path) 

C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english\snapshots\714eb0fa89d2f80546fda750413ed43d93601a13\config.json
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english\snapshots\714eb0fa89d2f80546fda750413ed43d93601a13\model.safetensors
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english\snapshots\714eb0fa89d2f80546fda750413ed43d93601a13\pytorch_model.bin
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english\snapshots\714eb0fa89d2f80546fda750413ed43d93601a13\rust_model.ot
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english\snapshots\714eb0fa89d2f80546fda750413ed43d93601a13\tf_model.h5
C:\Users\Gordts-De Laender\.cache\huggingface\hub\models--distilbert--distilbert-

[DistilBertForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification) is built upon the DistilBERT model and goes beyond the core DistilBERT model by adding a classification head on top of the encoder's output. This classification head takes the encoded representation of the sequence and transforms it into a probability distribution over different predefined classes. See [config.json](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english/blob/main/config.json) for the labels.

In [7]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline

tokenizer = DistilBertTokenizer.from_pretrained(model_id)
model = DistilBertForSequenceClassification.from_pretrained(model_id)

classifier = pipeline("text-classification", model=model, device=-1, tokenizer=tokenizer)
type(classifier) #To check which pipeline the classifier inherits 

transformers.pipelines.text_classification.TextClassificationPipeline

The following will print out the label and the corresponding confidence score (using the softmax function). The confidence score can be interpreted as a measure of how certain the model is about its prediction. For more info see [TextClassificationPipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)

## 2. Prompt engineering strategies

In [12]:
classifier("I don't like spaghetti")

[{'label': 'NEGATIVE', 'score': 0.9920676946640015}]

In [8]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad(): #Disabling gradient calculation improves efficiency for inference tasks.
    logits = model(**inputs).logits #The model processes the tokenized input and outputs a tensor named logits.logits represent the unnormalized scores (before applying softmax) for each potential sentiment class.

predicted_class_id = logits.argmax().item() #This corresponds to the class with the highest predicted probability.
model.config.id2label[predicted_class_id] #refers to the configuration of the pre-trained model see config.json)

'POSITIVE'

##### 2.1 adding instruction

In [8]:
sentiment_prompt = "What is the sentiment of the following text: "

In [9]:
pipeline(sentiment_prompt + "I don't like spaghetti")

[{'label': 'NEGATIVE', 'score': 0.994526207447052}]

##### 2.2 zero-shot 

##### 2.3 few-shot 

## 3. Stress-testing the model

##### 3.1 Inputs that should work well

##### 3.2 Failure-case inputs

##### 3.3 Test for consistency (looping???)

##### 3.4 Test for reliability (score, testing on the applied function to the model outputs in order to retrieve the scores,... )